# Target Propagation - Tutorial 2
## Differance Target Propagation

Tutorial on an improvement of target propagation that uses the difference between the propagated target and the reconstructed input. Also, will evaluate different learning rates for the output layer.

In [ ]:
import initialize

In [ ]:
from torchvision.datasets import FashionMNIST, CIFAR10, MNIST
from torchvision import transforms

from tools.training import train, classify, plot_loss_line
import torch
import math
import torch.nn as nn

from tools.modules import Sign, Sampler, Clamp

from tools.training import train, classify, plot_loss_line
import zenkai
from functools import partial

from tools.learners.target_propv2 import (
    BaselineLearner1, select_act, TPMultAltStepTheta, 
    NullTPStepTheta, LinearTPLearner, TPAltStepTheta, 
    DiffTPStepX, TPStepTheta, TPStepX, DeepLinearTPLearner,
    filter_acts, TPRegAltStepTheta
)


# Steps

1) Create each layer (Difference AutoencoderLearner)
2) Create the DiffTargetPropLearner
3) Run the training on the DiffTargetProp learner using varying learning rates

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))])


training_dataset = FashionMNIST(
    '../../Datasets/',
    transform=transform, 
    download=True
)
testing_dataset = FashionMNIST(
    '../../Datasets/',
    transform=transform, 
    download=True, 
    train=False
)

losses = {}
classifications = {}

epoch_results = {}
import math
k = math.prod(testing_dataset[0][0].shape)


In [ ]:

append = '3layer'
for key, act in [
    ('leaky_relu', nn.LeakyReLU), 
    ('clamp', Clamp), 
    # ('sigmoid', nn.Sigmoid),
    ('sign', partial(Sign, True))
]:

    # act = 'leaky_relu'
    # activation = nn.LeakyReLU
    # activation = partial(nn.LeakyReLU, 0, 1, False)

    pred = 'baseline'

    print(act)
    learner = BaselineLearner1(
        k, 200, 200, 200, 10, activation=act, lr=1e-3, dropout_p=0.1
    )
    cur_key = f'{pred}_{key}_{append}'
    losses[cur_key], epoch_results[cur_key] = train(learner, training_dataset, 40, device='cpu')
    classifications[cur_key] = classify(learner, testing_dataset)

In [ ]:

classifications

In [ ]:

it = 0


class Callback(object):

    def __init__(self, learner, total_epochs: int, cycle: int=10):

        self.learner = learner
        self.total_epochs = total_epochs
        self.it = 0
        self.temperature = 0.0
        self.increment = 1.0
        self.epoch = 0
        self.cycle = cycle

    def reset(self):

        self.it = 0
        self.epoch = 0

    def __call__(self, n_epochs, n_iterations, total_iterations):
        
        # if n_epochs > self.epoch:

        #     for act in filter_acts(learner, Sampler):
        #         act.temperature = min(
        #             act.temperature + self.increment, 1.0
        #         )
        #     self.epoch = n_epochs
        self.it = (self.it + 1) % self.cycle
        if self.it == (self.cycle // 2):
        # if self.it == self.cycle: # // 2):
            for step_theta in learner.step_thetas():
                step_theta.train_predictor = True
                step_theta.train_reconstruction = True
        elif self.it == 0:
            for step_theta in learner.step_thetas():
                step_theta.train_predictor = False
                step_theta.train_reconstruction = True


In [ ]:
from torch import nn

# epoch_results = {}

append = '3layer'

# for act in ['leaky_relu', 'sign', 'stochastic']:
for act, rec_weight in [
    # ('leaky_relu', None),
    # ('binary', None),
    # 'sampler', None),
    # ('clamp', None),
    ('sign', None),
    # ('sigmoid', None)
    # ('leaky_relu', 0.1),
    # ('sign', 0.1),
    # ('stochastic', 1.0),
    # ('stochastic', None),
]:
    print('Activation: ', act)

    key, act, in_act = select_act(act, rec_weight)

    key += f'_{append}'
    step_x = DiffTPStepX.factory(lr=1.0)

    step_theta = TPAltStepTheta.factory(
        zenkai.NNLoss('MSELoss', 'mean'),
        zenkai.NNLoss('MSELoss', 'mean'),
        1e-4, 1e-4, True, True, rec_weight, 1.0, # reg=1e0
    )

    i = 0
    out_x_lr = 1.0
    print(k)
    learner = DeepLinearTPLearner(
        k, 200, 200, 200, 10, step_theta, step_x, out_x_lr,
        act, act, in_act, True, False, 1e-4, dropout_p=None, 
        gaussian_noise=0.1, share_weights=False
    )

    callback = Callback(
        learner, 15, 2
    )

    losses[key], epoch_results[key] = train(
        learner, training_dataset, 40, 
        device='cpu', callback=callback, batch_size=128
    )
    classifications[key] = classify(learner, testing_dataset)

In [ ]:
classifications

In [ ]:

# learner = TargetPropLearner(
#     784, 300, 300, 300, 10, dropout_p=0.1, act=activation, in_act=in_act, out_x_lr=1e-3
# )
# alternator = AlternateTraining(learner, 1, 1)

# step_x = DiffTPStepX.factory(0.1)

# pred = 'baseline'

# step_theta = TPStepTheta.factory(
#     zenkai.NNLoss('MSELoss', 'mean'),
#     zenkai.NNLoss('MSELoss', 'mean'),
#     1e-3, True, True, 1.0, 1.0
# )

# i = 0
# out_x_lr = 0.1

# learner = DeepLinearTPLearner(
#     k, 300, 300, 300, 10, step_theta, step_x, 1e-3, 
#     nn.Tanh, nn.Tanh, Sign, True, True, 1e-3, 0.1, None
# )



# train(learner, training_dataset, 10, flatten=True, callback=None)

# for step_theta in learner.step_thetas():
#     step_theta.pred_weight = 1.0

# train(learner, training_dataset, 10, flatten=True, callback=None)

# step_theta = TPStepTheta.factory(
#     zenkai.NNLoss('MSELoss', 'mean'),
#     zenkai.NNLoss('MSELoss', 'mean'),
#     1e-3, True, True, 1.0, 1.0
# )


# learner = DiffConvTargetPropLearner(
#     1, (28, 28), 32, 32, 32, 10, 
#   kernel_size=(4, 4), stride=(2, 2), dropout_p=None, x_lr=0.1, out_x_lr=0.1, act=torch.nn.LeakyReLU,
#     reverse_act=torch.nn.LeakyReLU
# )

# classify(learner, testing_dataset)

# learner = DiffTargetPropLearner(
#     784, 300, 300, 300, 10, 1., 1.0, nn.Tanh, nn.Tanh, dropout_p=0.1, forward_in_act=Sign,
#     rec_weight=None, pred_weight=1.0, lr=1e-4, 
# )

# x = torch.rand(4, 1, 28, 28)
# learner(x)

In [ ]:
print(list(losses.keys()))

sub_losses = {
    k: v
    for k, v in losses.items() if k not in ('Target Prop - Leaky ReLU - 0.1', 'TargetProp - TanH - 0.1', 'Target Prop - Stochastic')
}
print(list(sub_losses.keys()))

In [ ]:
import numpy as np
def moving_average(a, n=3):
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n
    

sub_loss_moving = {}

for k, v in sub_losses.items():
    if k == 'TargetProp - TanH':
        k = 'TargetProp - Sign'

    sub_loss_moving[k] = moving_average(v, 10)




In [ ]:
plot_loss_line(
    list(sub_loss_moving.values()), list(sub_loss_moving.keys()), 
    'Training Loss', save_file='images/t2x2_diff_target_prop_3layer_2024_12_17_1.png'
)

In [ ]:
import pickle

all_results = {
    'losses': losses,
    'epoch_results': epoch_results,
    'classifications': classifications
}

with open('results/t2x2_3layer_12_17_2024.pkl', 'wb') as file:
    pickle.dump(all_results, file)



In [ ]:
classifications

In [ ]:


for k, v in all_results['epoch_results'].items():
    # if k in ['Target Prop - Leaky ReLU_3layer', 'Target Prop - Clamp_3layer', 'TargetProp - Sign_3layer']:
    print(v.keys())
    for k2, v2 in v.items():
        if k2 in ['loss']:

            print(k, k2, np.var(v2))


In [ ]:
import pickle

with open('results/t2x2_2layer_12_16_2024.pkl', 'rb') as file:
    epoch_results = pickle.load(file)





In [ ]:
list(epoch_results['Target Prop - Leaky ReLU'].keys())

In [ ]:
import pandas as pd
import numpy as np
list(epoch_results['Target Prop - Leaky ReLU'].keys())


sims = {}
for k, v in epoch_results.items():
    if k == 'baseline_leaky_relu':
        continue
    for sim in ['0_x_sim', '1_x_sim', '2_x_sim']:
        print(list(v.keys()))
        sims[f'{k}_{sim}'] = np.mean(v[sim])

sims


In [ ]:
# Don't propagate back reconstruction loss

# learner = DiffTargetPropLearner(
#     784, 300, 300, 300, 10, x_lr=0.5, act=torch.nn.LeakyReLU,
#     reverse_act=torch.nn.LeakyReLU
# )

# training_losses = train(learner, training_dataset, 20)
# classifications = classify(learner, testing_dataset)
# print(classifications)

In [ ]:
# learner = DiffTargetPropLearner(
#     784, 300, 300, 300, 10, x_lr=0.1, act=torch.nn.LeakyReLU,
#     reverse_act=torch.nn.LeakyReLU
# )

# training_losses2 = train(learner, training_dataset, 20)
# classifications2 = classify(learner, testing_dataset)
# print(classifications2)

In [ ]:

learner = DiffTargetPropLearner(
    784, 300, 300, 300, 10, x_lr=0.5, act=lambda: torch.sign, reverse_act=nn.Sigmoid, targetf=torch.sign
)

train(learner, training_dataset, 100)

In [ ]:


# learner = DiffTargetPropLearner(
#     784, 300, 300, 300, 10, x_lr=x_lr, 
#     act=torch.nn.LeakyReLU,
#     reverse_act=torch.nn.LeakyReLU
# )

# train(learner, training_dataset, 100)
# classify(learner, testing_dataset)
